In [18]:
import sys
# This makes the gdc19 package avaialble
sys.path.append('../')
import gdc19
# Be sure to set your working directory!!!
# this is the path to the Google Drive folder
gdc19.setup_bane()

'/Volumes/GoogleDrive/My Drive/utah-forge'

In [19]:
import vtki
import omfvtk
import PVGeo

-----

# Initial Inspection

This notebook loads up all the data and works through an initial inspection.

In [20]:
proj = omfvtk.load_project(gdc19.get_omf_project_filename())
proj

(MultiBlock)0x146974f48

In [21]:
topo = proj['land_surface']
topo

(UnstructuredGrid)0x146974d08

# Integrated Scene

Now use `vtki` to create and integrated rendering of all the available data


In [22]:
p = vtki.BackgroundPlotter()
# p.add_bounds_axes()

In [23]:
p.add_mesh(proj['temperature'], name='temp', 
           cmap='coolwarm', point_size=10, 
           render_points_as_spheres=True, stitle='Temperature')
p.add_mesh(proj['negro_mag_fault'], name='negro_mag_fault',
           show_edges=False, color='blue', opacity=0.5)
p.add_mesh(proj['opal_mound_fault'], name='opal_mound_fault',
           show_edges=False, color='orange', opacity=0.5)

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x146983168

In [24]:
# We can update this cell to change the texture
p.add_mesh(topo, 
           texture='geo_aer', 
           show_edges=False, 
           name='topo', 
           reset_camera=False)

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x1469836a8

In [25]:
# add the temperature boundaries
p.add_mesh(proj['temp_175c'], name='temp_175c', 
           show_edges=False, color='aquamarine',
           style='wireframe')
p.add_mesh(proj['temp_225c'], name='temp_225c', 
           show_edges=False, color='firebrick',
           style='wireframe')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x1469837c8

In [26]:
# Add the shape file stuff
p.add_mesh(proj['boundary'], name='boundary', 
           color='azure', render_lines_as_tubes=False, line_width=10)

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x146983708

In [27]:
# Add the granitoid boundary
#--
# Lets run a PVGeo filter to extract the topo surface from
#    the granitoid surface
ext = PVGeo.grids.ExtractTopography(
        remove=True, # remove the inactive cells
        tolerance=10.0 # buffer around the topo surface
        ).Apply(proj['top_granitoid'], topo)
granitoid = ext
#--
# Now plot it!
p.add_mesh(granitoid, name='top_granitoid',
           show_edges=False, style='surface',
           color='grey', opacity=0.75, reset_camera=False,
          )

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x146983888

In [28]:
# _ = p.screenshot('../integrated_view.png')

In [29]:
import pandas as pd
well_locs = pd.read_csv(gdc19.get_well_path('well_location_from_earth_model.csv'))
well_locs.head()

,Well ID,x,y,z (land surface),depth (m)
0,12-35,339385.010,4264212.985,1880.635342,2133.6
1,13-10,337709.018,4260659.997,1796.263259,1601.3
2,14-2,339393.002,4262171.961,1903.453009,1808.5
3,24-36,341221.016,4263743.011,2034.161472,1819.7
4,25-15,337873.021,4258706.959,1827.029208,2287.5


In [30]:
foo = PVGeo.pointsToPolyData(well_locs[['x', 'y', 'z (land surface)']].values)
p.add_mesh(foo, name='well_locations', point_size=10, color='orange')



(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x10a3bfa68

In [31]:
idxs = []
for i, pt in enumerate(foo.points):
    if vtki.is_inside_bounds(pt,proj['boundary'].bounds):
        idxs.append(i)
wells_inside = well_locs.loc[idxs]
wells_inside

,Well ID,x,y,z (land surface),depth (m)
35,58-32,335380.766,4263040.829,1681.616587,2296.9
56,RHS-14,335798.986,4264089.986,1690.846576,137.5
67,TPC-12,334826.000,4262756.046,1661.196616,150.0
68,TPC-14,335446.971,4264046.019,1675.199038,145.0


In [32]:
#Wells in FORGE Region
well_5832 = vtki.PolyData('well_data/58-32/well_5832.vtk')
p.add_mesh(well_5832, color = 'yellow', name = '58-32')

well_RHS14 = vtki.PolyData('well_data/RHS-14/well_RHS14.vtk')
p.add_mesh(well_RHS14, color = 'yellow', name = 'RHS-14')

well_TPC12 = vtki.PolyData('well_data/TPC-12/well_TPC12.vtk')
p.add_mesh(well_TPC12, color = 'yellow', name = 'TPC-12')

well_TPC14 = vtki.PolyData('well_data/TPC-14/well_TPC14.vtk')
p.add_mesh(well_TPC14, color = 'yellow', name = 'TPC-14')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x13f6ce108

In [33]:
#Wells of significant Depth
well_1310 = vtki.PolyData('well_data/other_wells/well_1310.vtk')
p.add_mesh(well_1310, color = 'yellow', name = '13-10')

well_142 = vtki.PolyData('well_data/other_wells/well_142.vtk')
p.add_mesh(well_142, color = 'yellow', name = '14-2')

well_2436 = vtki.PolyData('well_data/other_wells/well_2436.vtk')
p.add_mesh(well_2436, color = 'yellow', name = '24-36')

well_2515 = vtki.PolyData('well_data/other_wells/well_2515.vtk')
p.add_mesh(well_2515, color = 'yellow', name = '25-15')

well_283 = vtki.PolyData('well_data/other_wells/well_283.vtk')
p.add_mesh(well_283, color = 'yellow', name = '28-3')

well_5221 = vtki.PolyData('well_data/other_wells/well_5221.vtk')
p.add_mesh(well_5221, color = 'yellow', name = '52-21')

well_543 = vtki.PolyData('well_data/other_wells/well_543.vtk')
p.add_mesh(well_543, color = 'yellow', name = '54-3')

well_7110 = vtki.PolyData('well_data/other_wells/well_7110.vtk')
p.add_mesh(well_7110, color = 'yellow', name = '71-10')

well_DEC35 = vtki.PolyData('well_data/other_wells/well_DEC35.vtk')
p.add_mesh(well_DEC35, color = 'yellow', name = 'DEC-35')

well_Mar1 = vtki.PolyData('well_data/other_wells/well_Mar1.vtk')
p.add_mesh(well_Mar1, color = 'yellow', name = '1-Mar')

well_Sept1 = vtki.PolyData('well_data/other_wells/well_Sept1.vtk')
p.add_mesh(well_Sept1, color = 'yellow', name = '1-Sept')

well_8233 = vtki.PolyData('well_data/other_wells/well_8233.vtk')
p.add_mesh(well_8233, color = 'yellow', name = '82-33')

well_Acord1 = vtki.PolyData('well_data/other_wells/well_Acord1.vtk')
p.add_mesh(well_Acord1, color = 'red', name = 'Acord-1')

well_GPC15 = vtki.PolyData('well_data/other_wells/well_GPC15.vtk')
p.add_mesh(well_GPC15, color = 'yellow', name = 'GPC-15')

well_OH1 = vtki.PolyData('well_data/other_wells/well_OH1.vtk')
p.add_mesh(well_OH1, color = 'yellow', name = 'OH-1')

well_OH4 = vtki.PolyData('well_data/other_wells/well_OH4.vtk')
p.add_mesh(well_OH4, color = 'yellow', name = 'OH-4')

well_OH5 = vtki.PolyData('well_data/other_wells/well_OH5.vtk')
p.add_mesh(well_OH5, color = 'yellow', name = 'OH-5')

well_OH7 = vtki.PolyData('well_data/other_wells/well_OH7.vtk')
p.add_mesh(well_OH7, color = 'yellow', name = 'OH-7')

well_OH8 = vtki.PolyData('well_data/other_wells/well_OH8.vtk')
p.add_mesh(well_OH8, color = 'yellow', name = 'OH-8')

well_353 = vtki.PolyData('well_data/other_wells/well_353.vtk')
p.add_mesh(well_353, color = 'yellow', name = '35-3')


(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x13f6cefa8

In [35]:
#Wells of insignificant depth
well_BigCedar = vtki.PolyData('well_data/other_wells/well_BigCedar.vtk')
p.add_mesh(well_BigCedar, color = 'yellow', name = 'Big Cedar')

#Pure Granitoid
well_EV4113 = vtki.PolyData('well_data/other_wells/well_EV4113.vtk')
p.add_mesh(well_EV4113, color = 'green', name = 'EV-4113')

well_EV4115 = vtki.PolyData('well_data/other_wells/well_EV4115.vtk')
p.add_mesh(well_EV4115, color = 'green', name = 'EV-4115')

well_GPC10 = vtki.PolyData('well_data/other_wells/well_GPC10.vtk')
p.add_mesh(well_GPC10, color = 'green', name = 'GPC-10')

well_GPC9 = vtki.PolyData('well_data/other_wells/well_GPC9.vtk')
p.add_mesh(well_GPC9, color = 'green', name = 'GPC-9')

well_MM101 = vtki.PolyData('well_data/other_wells/well_MM101.vtk')
p.add_mesh(well_MM101, color = 'green', name = 'MM-101')

well_PPCDR = vtki.PolyData('well_data/other_wells/well_PPCDR.vtk')
p.add_mesh(well_PPCDR, color = 'green', name = 'PPC-DR')

well_PPCNR = vtki.PolyData('well_data/other_wells/well_PPCNR.vtk')
p.add_mesh(well_PPCNR, color = 'green', name = 'PPC-NMR')

well_PPC2 = vtki.PolyData('well_data/other_wells/well_PPC2.vtk')
p.add_mesh(well_PPC2, color = 'green', name = 'PPC-2')

well_PPC5 = vtki.PolyData('well_data/other_wells/well_PPC5.vtk')
p.add_mesh(well_PPC5, color = 'green', name = 'PPC-5')

well_PPC6 = vtki.PolyData('well_data/other_wells/well_PPC6.vtk')
p.add_mesh(well_PPC6, color = 'green', name = 'PPC-6')

well_RHS189 = vtki.PolyData('well_data/other_wells/well_RHS189.vtk')
p.add_mesh(well_RHS189, color = 'green', name = 'RHS-189')

well_UUHF3 = vtki.PolyData('well_data/other_wells/well_UUHF3.vtk')
p.add_mesh(well_UUHF3, color = 'green', name = 'UUHF-3')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x13f6e4dc8